In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [8]:
class ShipCharter(Agent):
    """An agent representing a ship charter."""
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.fiat_balance = 10000  # Starting fiat balance

    def step(self):
        # Logic for sending money to agents
        if self.fiat_balance > 0:
            agent = self.random.choice(self.model.ship_agents)
            amount = random.randint(1000, 5000)  # Random amount to send
            self.fiat_balance -= amount
            agent.receive_fiat(amount)

In [9]:
class ShipAgent(Agent):
    """An agent representing a ship agent."""
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.fiat_balance = 0
        self.token_balance = 0

    def receive_fiat(self, amount):
        self.fiat_balance += amount
        # Logic for exchanging fiat for tokens
        self.exchange_fiat_for_tokens(amount)

    def exchange_fiat_for_tokens(self, amount):
        tokens_received = amount  # Assuming 1:1 exchange for simplicity
        self.token_balance += tokens_received

In [10]:
class ShipSupplier(Agent):
    """An agent representing a ship supplier."""
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.token_balance = 0
        self.fiat_balance = 0

    def receive_payment(self, amount):
        self.token_balance += amount

    def redeem_tokens(self):
        # Logic for redeeming tokens for fiat
        fee = self.model.transaction_fee  # Get the transaction fee from the model
        if self.token_balance > 0:
            tokens_to_redeem = self.token_balance
            redeem_amount = tokens_to_redeem - fee  # Deduct fee from redemption
            if redeem_amount > 0:
                self.fiat_balance += redeem_amount
                self.token_balance = 0  # All tokens redeemed
                self.model.total_fees_collected += fee  # Track the collected fees
            else:
                print(f"Insufficient tokens after fee for agent {self.unique_id}")

In [17]:
class StablecoinModel(Model):
    """A model with some number of agents."""
    
    def __init__(self, N, transaction_fee):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.transaction_fee = transaction_fee  # Set the transaction fee
        self.total_fees_collected = 0  # Initialize total fees collected

        # Create lists to hold different agent types
        self.ship_agents = []  # List to store ShipAgent instances
        
        # Create agents
        for i in range(N // 3):
            a = ShipCharter(i, self)
            self.schedule.add(a)
        
        for i in range(N // 3, 2 * N // 3):
            a = ShipAgent(i, self)
            self.ship_agents.append(a)  # Append to ship_agents list
            self.schedule.add(a)

        for i in range(2 * N // 3, N):
            a = ShipSupplier(i, self)
            self.schedule.add(a)

    def step(self):
        self.schedule.step()

In [18]:
# Example of running the model with a transaction fee
transaction_fee = 100  # Set a fixed transaction fee for offramps
model = StablecoinModel(30, transaction_fee)

In [19]:
# Simulate a few steps to see transactions and fees collected
for i in range(100):  # Run for 100 steps
    model.step()

# Output total fees collected after the simulation
print(f"Total transaction fees collected: {model.total_fees_collected}")

Total transaction fees collected: 0


In [31]:
model.schedule.time

100